In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
img_yes = '../data/yes/'
img_no = '../data/no/'

yes_files = os.listdir(img_yes)
no_files = os.listdir(img_no)

yes_images, no_images = [], []
print('[!] Reading files with positive diagnostic...')
for file in yes_files:
    yes_images.append(plt.imread(f'{img_yes}{file}'))
print('[!] Positive files read!')

print('[!] Reading files with negative diagnostic...')
for file in no_files:
    no_images.append(plt.imread(f'{img_no}{file}'))
print('[!] Negative files read!')

all_images = yes_images + no_images
YES_SIZE, NO_SIZE = len(yes_images), len(no_images)
TOTAL_SIZE = len(all_images)

[!] Reading files with positive diagnostic...
[!] Positive files read!
[!] Reading files with negative diagnostic...
[!] Negative files read!


In [3]:
THRESH = 30
MAX_VALUE = 255

def convert_to_int_matrix(img_list):
    MIN, MAX = 0, 255
    for i, img in enumerate(img_list):
        if img.dtype == np.float32:
            tmp = np.copy(img) * MAX
            img_list[i] = tmp.astype(np.uint8)

def resize_imgs(img_list, height, width):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.resize(img, (height, width))

def imgs_to_grayscale(img_list):
    for i, img in enumerate(img_list):
        if len(img.shape) == 3:
            img_list[i] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def gaussian_blur(img_list):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.GaussianBlur(img, (5, 5), 0) 

def threshold_imgs(img_list):
    for i, img in enumerate(img_list):
        _, img_list[i] = cv2.threshold(img, THRESH, MAX_VALUE, cv2.THRESH_BINARY)

def erode_imgs(img_list, iter=1):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.erode(img, None, iterations=iter)

def dilate_imgs(img_list, iter=1):
    for i, img in enumerate(img_list):
        img_list[i] = cv2.dilate(img, None, iterations=iter)

def flatten_contour(contour_list):
    tmp = contour_list[0]
    shape = tmp.shape
    aux = np.zeros(shape=(shape[0], shape[2]), dtype=int)
    for i in range(shape[0]):
        aux[i] = tmp[i].flatten()
    return aux

def find_extreme_points(contour_list):
    col_1 = contour_list[:, 0]
    col_0 = contour_list[:, 1]
    x_left, x_right = np.argmin(col_0), np.argmax(col_0)
    y_top, y_bot = np.argmax(col_1), np.argmin(col_1)
    ret = np.zeros(shape=(4, 2), dtype=int)
    ret[0, :], ret[1, :] = contour_list[x_left], contour_list[x_right]
    ret[2, :], ret[3, :] = contour_list[y_top], contour_list[y_bot]
    return ret  

def get_contour(img_list_morph, img_list, mode):
    extreme_points, i = [], 0
    for img_morph, img in zip(img_list_morph, img_list):
        cont, _ = cv2.findContours(image=img_morph, mode=mode, method=cv2.CHAIN_APPROX_NONE)
        cont_flat = flatten_contour(cont)
        extreme_points.append(find_extreme_points(cont_flat))
        i+= 1
        if i % 50 == 0: print(f'[!] {i} images processed...')
        # print(extreme_points[-1])
        img_copy = img.copy()
        # print(f'img_shape: {img_copy.shape}')
        # print(f'extreme_point: {extreme_points[-1]}')
        # for i in range(4):
        #     img_copy = cv2.circle(img_copy, extreme_points[-1][i, :], radius=5, color=(0, 255, 0), thickness=-1)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
        # cv2.drawContours(image=img_copy, contours=cont, contourIdx=-1, color=(0, 0, 255), thickness=4, lineType=cv2.LINE_AA)
        # cv2.imshow("Image", img_copy)
        # cv2.waitKey()
    print(f'[!] Total of {i} images processed!')
    return np.array(extreme_points)  

def draw_img(img_list, index):
    if index < len(img_list):
        cv2.imshow("Brain image", img_list[index])
        cv2.waitKey()

In [4]:
INDEX = 0
HEIGHT, WIDTH = 214, 214
convert_to_int_matrix(all_images)
all_images_morph = all_images.copy()
draw_img(all_images_morph, INDEX)
imgs_to_grayscale(all_images_morph)
draw_img(all_images_morph, INDEX)  
threshold_imgs(all_images_morph)
draw_img(all_images_morph, INDEX)
erode_imgs(all_images_morph, iter=2)
draw_img(all_images_morph, INDEX)
dilate_imgs(all_images_morph, iter=4)
draw_img(all_images_morph, INDEX)
gaussian_blur(all_images_morph)
draw_img(all_images_morph, INDEX)
cv2.destroyAllWindows()
extreme_points = get_contour(all_images_morph, all_images, cv2.RETR_EXTERNAL)

[!] 50 images processed...
[!] 100 images processed...
[!] 150 images processed...
[!] 200 images processed...
[!] 250 images processed...
[!] Total of 253 images processed!


In [5]:
def cut_images(extreme_points, img_list):
    new_images = []
    for extreme_point, img in zip(extreme_points, img_list):
        x_left, x_right = extreme_point[0, 1], extreme_point[1, 1]
        y_top, y_bot = extreme_point[2, 0], extreme_point[3, 0]
        size_h, size_v = x_right - x_left, y_top - y_bot
        # print(f'img_shape: {img.shape}')
        # print(f'x_left: {x_left} - x_right: {x_right} - y_top: {y_top} - y_bot: {y_bot}')
        # tmp = np.zeros(shape=(size_h, size_v), dtype=np.uint8)
        # tmp = img[x_left:x_right, y_bot:y_top].copy()
        tmp = np.zeros(shape=(size_h, size_v), dtype=np.uint8)
        for i, k in zip(range(x_left, x_right), range(size_h)):
            for j, l in zip(range(y_bot, y_top), range(size_v)):
                tmp[k, l] = img[i, j]
        new_images.append(tmp)
    return new_images

In [6]:
grey_imgs = all_images.copy()
imgs_to_grayscale(grey_imgs)
new_images = cut_images(extreme_points, grey_imgs)

In [7]:
def check_images(img_morph_list, img_list):
    correct_yes_images, correct_no_images = [], []
    incorrect_yes_images, incorrect_no_images = [], []
    for img_morph, img, i in zip(img_morph_list, img_list, range(len(img_list))):
        cv2.imshow("Brain", img_morph)
        key = cv2.waitKey()
        if key == 13:
            if i < YES_SIZE:
                correct_yes_images.append(img_morph)
            else:
                correct_no_images.append(img_morph)
        elif key == 8:
            if i < YES_SIZE:
                incorrect_yes_images.append(img)
            else:
                incorrect_no_images.append(img)
        if i % 50 == 0:
            print(f'[!] {i} images checked...')
    return correct_yes_images, correct_no_images, incorrect_yes_images, incorrect_no_images

In [8]:
correct_yes_images, correct_no_images, incorrect_yes_images, incorrect_no_images = check_images(new_images, grey_imgs)
cv2.destroyAllWindows()
resize_imgs(correct_yes_images, HEIGHT, WIDTH)
resize_imgs(correct_no_images, HEIGHT, WIDTH)
# correct_yes_images = np.load('./numpy_arrays/correct_yes_images.npy', allow_pickle=True)
# correct_no_images = np.load('./numpy_arrays/correct_no_images.npy', allow_pickle=True)
# incorrect_yes_images = np.load('./numpy_arrays/incorrect_yes_images.npy', allow_pickle=True)
# incorrect_no_images = np.load('./numpy_arrays/incorrect_no_images.npy', allow_pickle=True)

[!] 0 images checked...
[!] 50 images checked...
[!] 100 images checked...
[!] 150 images checked...
[!] 200 images checked...
[!] 250 images checked...


In [9]:
print(f'[!] {len(correct_yes_images) + len(correct_no_images)} correct images!')
print(f'[!] {len(incorrect_yes_images) + len(incorrect_no_images)} incorrect images!')

[!] 205 correct images!
[!] 48 incorrect images!


In [10]:
NP_PATH = './numpy_arrays'
np.save(f'{NP_PATH}/correct_yes_images.npy', correct_yes_images, allow_pickle=True)
np.save(f'{NP_PATH}/correct_no_images.npy', correct_no_images, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_yes_images.npy', incorrect_yes_images, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_no_images.npy', incorrect_no_images, allow_pickle=True)

c:\Users\kmart\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\lib\npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [11]:
THRESH, INDEX = 20, 0
YES_SIZE, NO_SIZE = len(incorrect_yes_images), len(incorrect_no_images)

original_incorrect_images = np.concatenate((incorrect_yes_images, incorrect_no_images))
all_incorrect_images = original_incorrect_images.copy()

threshold_imgs(all_incorrect_images)
erode_imgs(all_incorrect_images, iter=2)
dilate_imgs(all_incorrect_images, iter=4)
gaussian_blur(all_incorrect_images)
cv2.destroyAllWindows()

extreme_points = get_contour(all_incorrect_images, original_incorrect_images, cv2.RETR_EXTERNAL)
new_images = cut_images(extreme_points, original_incorrect_images)
res_imgs = check_images(new_images, original_incorrect_images)
cv2.destroyAllWindows()

print(f'[!] {len(res_imgs[0]) + len(res_imgs[1])} correct images!')
print(f'[!] {len(res_imgs[2]) + len(res_imgs[3])} incorrect images!')

<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


[!] Total of 48 images processed!
[!] 0 images checked...
[!] 17 correct images!
[!] 31 incorrect images!


In [12]:
resize_imgs(res_imgs[0], HEIGHT, WIDTH)
resize_imgs(res_imgs[1], HEIGHT, WIDTH)
res_correct_yes = np.concatenate((np.load(f'{NP_PATH}/correct_yes_images.npy'), np.array(res_imgs[0], dtype=np.uint8)))
res_correct_no = np.concatenate((np.load(f'{NP_PATH}/correct_no_images.npy'), np.array(res_imgs[1], dtype=np.uint8)))
print(f'[!] {len(res_correct_yes) + len(res_correct_no)} total correct images!')
print(f'[!] {TOTAL_SIZE - (len(res_correct_yes)+len(res_correct_no))} total incorrect images!')
np.save(f'{NP_PATH}/correct_yes_images_2.npy', res_correct_yes, allow_pickle=True)
np.save(f'{NP_PATH}/correct_no_images_2.npy', res_correct_no, allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_yes_images_2.npy', res_imgs[2], allow_pickle=True)
np.save(f'{NP_PATH}/incorrect_no_images_2.npy', res_imgs[3], allow_pickle=True)

[!] 222 total correct images!
[!] 31 total incorrect images!


In [13]:
class_yes = np.ones(shape=(len(res_correct_yes,)), dtype=np.uint8)
class_no = np.zeros(shape=(len(res_correct_no)), dtype=np.uint8)
np.save(f'{NP_PATH}/classes.npy', np.concatenate((class_yes, class_no)), allow_pickle=True)

In [14]:
print(len(res_correct_yes))
print(len(res_correct_no))
for img in res_correct_yes:
    cv2.imshow("yes", img)
    cv2.waitKey()
cv2.destroyAllWindows()
for img in res_correct_no:
    cv2.imshow("no", img)
    cv2.waitKey()
cv2.destroyAllWindows()
# NP_PATH = './numpy_arrays/'
# THRESH, INDEX = 30, 0
# remaining_incorrect_yes = np.load(f'{NP_PATH}/incorrect_yes_images_2.npy', allow_pickle=True)
# remaining_incorrect_no = np.load(f'{NP_PATH}/incorrect_no_images_2.npy', allow_pickle=True)

# all_remaining_images = np.concatenate((remaining_incorrect_yes, remaining_incorrect_no))
# all_reamaining_images_morph = all_remaining_images.copy()
# threshold_imgs(all_reamaining_images_morph)
# draw_img(all_reamaining_images_morph, INDEX)
# erode_imgs(all_reamaining_images_morph, iter=7)
# draw_img(all_reamaining_images_morph, INDEX)
# dilate_imgs(all_reamaining_images_morph, iter=7)
# draw_img(all_reamaining_images_morph, INDEX)
# gaussian_blur(all_reamaining_images_morph)
# draw_img(all_reamaining_images_morph, INDEX)
# cv2.destroyAllWindows()

# extreme_points = get_contour(all_reamaining_images_morph, all_remaining_images, cv2.RETR_EXTERNAL)
# new_images = cut_images(extreme_points, all_remaining_images)
# res_imgs = check_images(new_images, all_remaining_images)
# cv2.destroyAllWindows()

# print(f'[!] {len(res_imgs[0]) + len(res_imgs[1])} correct images!')
# print(f'[!] {len(res_imgs[2]) + len(res_imgs[3])} incorrect images!')

135
87
